In [70]:
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
from imblearn.over_sampling import RandomOverSampler

# Make train and test data
ros = RandomOverSampler(random_state=42)


data_train = pd.read_csv("hospital_deaths_train.csv")
X_train = data_train.drop("In-hospital_death", axis=1)
y_train = data_train['In-hospital_death'].to_numpy()

data_test = pd.read_csv("hospital_deaths_test.csv")
X_test = data_test.drop("In-hospital_death", axis=1)
y_test = data_test['In-hospital_death'].to_numpy()

means = X_train.mean()

X_train = X_train.fillna(means).to_numpy()

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

X_resampled = X_resampled.reshape(X_resampled.shape[0], 1, X_resampled.shape[1])
y_resampled = y_resampled.reshape(-1, 1, 1)

X_test = X_test.fillna(means).to_numpy()

X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
y_test = y_test.reshape(-1, 1, 1)

In [71]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer((1, 115)),
    tf.keras.layers.Dense(5, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer='adam', loss='binary_crossentropy')

model.fit(X_resampled, y_resampled, epochs=5, batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)

f1 = f1_score(y_test.flatten(), y_pred_binary.flatten())
print(f"F2 score: {f1:.4f}")

# Overfitted data and nothing good

Epoch 1/5
175/175 [==============================] - 1s 1ms/step - loss: 0.7109
Epoch 2/5
175/175 [==============================] - 0s 1ms/step - loss: 0.6954
Epoch 3/5
175/175 [==============================] - 0s 1ms/step - loss: 0.6935
Epoch 4/5
175/175 [==============================] - 0s 1ms/step - loss: 0.6933
Epoch 5/5
24/24 [==============================] - 0s 831us/step
F2 score: 0.2415
